In [1]:

import process_data


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

engagements_raw, demographics_raw, followers_raw, top_posts_raw = process_data.read_data()
engagements, engagements_by_day = process_data.preprocess_engagements(engagements_raw)
demographics = process_data.preprocess_demographics(demographics_raw)
followers, total_followers, monthly_followers = process_data.preprocess_followers(followers_raw)
top_posts_preview_with_topics = process_data.preprocess_topposts(top_posts_raw)

In [2]:
top_posts_preview_with_topics

,Unnamed: 0,Post URL,Post publish date,Engagements,Impressions,Title,Description,Thumbnail,Topics
0,0,https://www.linkedin.com/feed/update/urn:li:ac...,7/17/2023,835,7548,KT Moore on LinkedIn: #honoredandgrateful #dei...,I want to express my heartfelt thanks to SEMI ...,https://media.licdn.com/dms/image/D5610AQGkQ-Y...,"DEIB Recognition, Gratitude, SEMI Honor, Shapi..."
1,1,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,530,57830,KT Moore on LinkedIn: How Artificial Intellige...,Behind every Formula One (#F1) car tearing up ...,https://media.licdn.com/dms/image/sync/D5627AQ...,"Engineering Excellence, Team Collaboration, Hi..."
2,2,https://www.linkedin.com/feed/update/urn:li:ac...,12/24/2023,511,10631,KT Moore on LinkedIn: #holidayseason #happyhol...,"As the year comes to a close, let&#39;s expres...",https://media.licdn.com/dms/image/D5622AQGAiid...,"Gratitude, Reflection, Growth, Appreciation, N..."
3,3,https://www.linkedin.com/feed/update/urn:li:ac...,6/1/2023,405,8802,KT Moore on LinkedIn: #innovation #intelligent...,Excited to be part of this morning’s opening b...,https://media.licdn.com/dms/image/D5622AQEhpM_...,"Nasdaq Ceremony, CEO Devgan, Trading, Cadence ..."
4,4,https://www.linkedin.com/feed/update/urn:li:ac...,11/23/2023,337,10124,KT Moore on LinkedIn: #thanksgiving2023 #thank...,"This Thanksgiving, I encourage you to not only...",https://media.licdn.com/dms/image/D5610AQEH9et...,"Gratitude, Community, Compassion, Inclusivity,..."
5,5,https://www.linkedin.com/feed/update/urn:li:ac...,1/1/2024,291,5087,KT Moore on LinkedIn: #happynewyear | 10 comments,Wishing everyone in and out of my network a #H...,https://media.licdn.com/dms/image/D5622AQHIdoL...,"New Beginnings, Growth, Innovation, Reflection..."
6,6,https://www.linkedin.com/feed/update/urn:li:ac...,11/11/2023,268,6182,KT Moore on LinkedIn: #veteransday | 20 comments,"Having grown up in a military family, #Veteran...",https://media.licdn.com/dms/image/D5622AQEVjKX...,"Military Family, Veterans Day, Reflection, Gra..."
7,7,https://www.linkedin.com/feed/update/urn:li:ac...,6/26/2023,233,18304,KT Moore on LinkedIn: Driving Intelligence in ...,"In a recent interview with EE Times Asia, Mich...",https://media.licdn.com/dms/image/sync/D5627AQ...,"Cadence Interview, Sales Strategy, Asia Pacifi..."
8,8,https://www.linkedin.com/feed/update/urn:li:ac...,1/17/2024,215,16195,KT Moore on LinkedIn: AMD&#39;s Next GPU Is a ...,Building the MI300 is akin to building Skyscra...,https://media.licdn.com/dms/image/sync/D4D27AQ...,"MI300, Skyscrapers, Silicon, Co-optimization, ..."
9,9,https://www.linkedin.com/feed/update/urn:li:ac...,5/4/2023,207,7797,KT Moore on LinkedIn: CadenceLIVE 2023 Satelli...,Out of all the amazing presentations that I&#3...,https://media.licdn.com/dms/image/sync/D5627AQ...,"Innovation, Inspiration, Technology, Leadershi..."


In [4]:
%load_ext autoreload
%autoreload 2

from dash import Dash, html, dcc, callback_context
import plotly.express as px
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import callbacks
import process_data
from forecasting import create_forecast_layout, create_forecast_callbacks
from articles import create_articles_layout, create_articles_callbacks

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

engagements_raw, demographics_raw, followers_raw, top_posts_raw = process_data.read_data()
engagements, engagements_by_day = process_data.preprocess_engagements(engagements_raw)
demographics = process_data.preprocess_demographics(demographics_raw)
followers, total_followers, monthly_followers = process_data.preprocess_followers(followers_raw)
top_posts_preview_with_topics = process_data.preprocess_topposts(top_posts_raw)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://fonts.googleapis.com/css2?family=Open+Sans:wght@300;400;600;700&display=swap'])
app.title = 'LinkedIn Analytics Dashboard'

# Define color scheme
colors = {
    'primary': '#0077B5',  # LinkedIn blue
    'secondary': '#00A0DC',
    'background': '#F3F6F8',
    'text': '#283E4A',
    'accent': '#7FC15E'
}

app.layout = dbc.Container([ 
    
    dbc.Row([
        dbc.Col(html.Img(src='/assets/img/Logo1.png', className='app-logo'), width=2),
        dbc.Col(html.H1("LinkedIn Analytics Dashboard", className='dashboard-title'), width=10)
    ], className='header'),

    dbc.Tabs([
        dbc.Tab(label='Overview',  tab_id='overview', tabClassName='custom-tab', activeTabClassName='custom-tab-active', children=[
        dbc.Row([
            dbc.Col([
                html.H4("Select Date Range:", className='date-label'),
                dcc.DatePickerRange(
                    id='date-picker-range',
                    min_date_allowed=engagements['Date'].min(),
                    max_date_allowed=engagements['Date'].max(),
                    initial_visible_month=engagements['Date'].max(),
                    start_date=engagements['Date'].min(),
                    end_date=engagements['Date'].max(),
                    className='date-picker'
                )
            ], width=12)
        ], className='date-range-row'),
        
        dbc.Row([
            dbc.Col(html.H2("Overview", className='section-title'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col(dbc.Card(id='impressions', className='metric-card'), width=2),
            dbc.Col(dbc.Card(id='engagements', className='metric-card'), width=2),
            dbc.Col(dbc.Card(id='engagement_rate', className='metric-card'), width=2),
            dbc.Col(dbc.Card(id='emv_value', className='metric-card'), width=2),
            dbc.Col(dbc.Card([
                html.Div('Followers', className='metric-title'),
                html.Div(html.B(f"{total_followers:,.0f}", className='metric-value'))
            ], className='metric-card'), width=2)
        ], className='metrics-row'),
        
        dbc.Row([
            dbc.Col(html.H2("Follower Growth", className='section-title'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col(dcc.Graph(
                figure=px.line(monthly_followers, x='YearMonth', y='New followers', 
                            title='Monthly Follower Growth',
                            labels={'YearMonth': 'Month', 'New followers': 'New Followers'},
                            color_discrete_sequence=[colors['primary']])
            ), width=12)
        ]),
        
        dbc.Row([
            dbc.Col(html.H2("Audience Demographics", className='section-title'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                    id='audience-dropdown',
                    options=[
                        {'label': 'Job titles', 'value': 'Job titles'},
                        {'label': 'Locations', 'value': 'Locations'},
                        {'label': 'Industries', 'value': 'Industries'},
                        {'label': 'Seniority', 'value': 'Seniority'},
                        {'label': 'Company size', 'value': 'Company size'},
                        {'label': 'Companies', 'value': 'Companies'},
                    ],
                    value='Job titles',
                    className='audience-dropdown'
                )
            ], width=3),
            dbc.Col(dcc.Graph(id='audience-graph'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col(html.H2("Engagement Analysis", className='section-title'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                    id='variable-dropdown',
                    options=[
                        {'label': 'Engagements', 'value': 'Engagements'},
                        {'label': 'Impressions', 'value': 'Impressions'},
                        {'label': 'Engagement Rate', 'value': 'EngagementRate'},
                    ],
                    value='Engagements',
                    className='variable-dropdown'
                )
            ], width=3),
            dbc.Col(dcc.Graph(id='engagements-graph'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H3('What is the best day of week to post?', className='optimal-day-title'),
                    html.Div(id='best-day', className='optimal-day-value'),
                    html.Div(id='optimal-days', className='optimal-day-description')
                ], className='optimal-day-container')
            ], width=3),
            dbc.Col(dcc.Graph(id='optimal-graph'), width=9, ),

        ], className='optimal-day-row'),
        
        dbc.Row([
            dbc.Col(html.H2("Top Posts", className='section-title'), width=12)
        ]),
        
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                    id='posts-dropdown',
                    options=[
                        {'label': 'Top engaging posts', 'value': 'Top engaging posts'},
                        {'label': 'Top posts by reach', 'value': 'Top posts by reach'},
                    ],
                    value='Top engaging posts',
                    className='posts-dropdown'
                )
            ], width=3)
        ]),
        
        dbc.Row([
            dbc.Col(
                html.Div(id='top-posts-table-container'),
                width=12
            )
        ])
        ]),
        dbc.Tab(label="Forecasting",  tab_id='forecasting', tabClassName='custom-tab', activeTabClassName='custom-tab-active', children=[
            create_forecast_layout()
        ]),
        dbc.Tab(label="Articles Analysis", tab_id='articles', tabClassName='custom-tab', activeTabClassName='custom-tab-active', children=[
            create_articles_layout()
        ])
    ], id='dashboard-tabs', active_tab='overview', className='custom-tabs'),
    
], fluid=True, className='dashboard-container')

@app.callback(
    Output('engagements', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_engagements(start_date, end_date):
    return callbacks.update_engagements(start_date, end_date)

@app.callback(
    Output('impressions', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)    
def update_impressions(start_date, end_date):
    return callbacks.update_impressions(start_date, end_date)

@app.callback(
    Output('engagement_rate', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
) 
def update_engagements_rate(start_date, end_date):
    return callbacks.update_engagements_rate(start_date, end_date)

@app.callback(
    Output('emv_value', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)   
def update_emv(start_date, end_date):   
    return callbacks.update_emv(start_date, end_date)

@app.callback(
    Output('audience-graph', 'figure'),
    [Input('audience-dropdown', 'value')]
)
def update_audience_graph(selected_variable):
    return callbacks.update_audience_graph(selected_variable)

@app.callback(
    Output('engagements-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_engagements_graph(start_date, end_date, selected_variable):
    return callbacks.update_engagements_graph(start_date, end_date, selected_variable)

@app.callback(
    Output('optimal-graph', 'figure'),
    Output('best-day', 'children'),
    Output('optimal-days', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_optimal_graph(start_date, end_date, selected_variable):
    return callbacks.update_optimal_graph(start_date, end_date, selected_variable)

@app.callback(
    Output('top-posts-table-container', 'children'),
    [Input('posts-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_top_posts_table(selected_variable, start_date, end_date):
    filtered_data = top_posts_preview_with_topics[
        (top_posts_preview_with_topics['Post publish date'] >= start_date) & 
        (top_posts_preview_with_topics['Post publish date'] <= end_date)
    ]
    
    if selected_variable == 'Top engaging posts':
        data = filtered_data.sort_values(by='Engagements', ascending=False).head(5)
    else:
        data = filtered_data.sort_values(by='Impressions', ascending=False).head(5)
    
    table_header = [
        html.Thead(html.Tr([
            html.Th("Published on"),
            html.Th("Posts"),
            html.Th("Description"),
            html.Th("Impressions"),
            html.Th("Reactions"),
            html.Th("Post Link"),
            html.Th("Topics")
        ]))
    ]

    table_body = [html.Tbody([
        html.Tr([
            html.Td(row['Post publish date']),
            html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else "No thumbnail"),
            html.Td(row['Description']),
            html.Td(f"{row['Impressions']:,}"),
            html.Td(f"{row['Engagements']:,}"),
            html.Td(html.A("View on LinkedIn", href=row['Post URL'], target="_blank")),
            html.Td(row['Topics'])
        ]) for _, row in data.iterrows()
    ])]

    return dbc.Table(table_header + table_body, bordered=True, hover=True, responsive=True, striped=True, className='top-posts-table')

# Create callbacks for forecasting and articles
create_forecast_callbacks(app)
create_articles_callbacks(app)

# Add CSS styles
app.css.append_css({
    "external_url": "/assets/styles.css"
})

if __name__ == '__main__':
    app.run_server(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%load_ext autoreload
%autoreload 2

from dash import Dash, html, dcc, callback_context
import plotly.express as px
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import callbacks
import process_data


import warnings
warnings.filterwarnings("ignore", category=UserWarning)


engagements_raw, demographics_raw, followers_raw, top_posts_raw = process_data.read_data()
engagements, engagements_by_day = process_data.preprocess_engagements(engagements_raw)
demographics = process_data.preprocess_demographics(demographics_raw)
followers, total_followers, monthly_followers = process_data.preprocess_followers(followers_raw)
top_posts_preview_with_topics = process_data.preprocess_topposts(top_posts_raw)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'LinkedIn Analytics'


app.layout = html.Div([
        
        html.Img(
            src='/assets/img/Logo1.png',
            className='app-logo'
        ),
        
        html.H1(
            "LinkedIn Analytics Dashboard",
            className='title'
        ),
        dcc.Markdown(
            '**Choose a date range:** ',
            className='date-cta',
            ),
        dcc.DatePickerRange(
            id='date-picker-range',
            min_date_allowed=engagements['Date'].min(),
            max_date_allowed=engagements['Date'].max(),
            initial_visible_month=engagements['Date'].max(),
            start_date=engagements['Date'].min(),
            end_date=engagements['Date'].max(),
            className='date-picker',
            ),
        dbc.Row(
            [
            dcc.Markdown('**Overview** ', className='heading1'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/impression.png', className='icon')
                ),
                html.Div(id='impressions', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/engagements-icon.png', className='icon')
                ),
                html.Div(id='engagements', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/engagement.png', className='icon')
                ),
                html.Div(id='engagement_rate', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/like.png', className='icon')
                ),
                html.Div(id='emv_value', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/followers.png', className='icon')
                ),
                html.Div(id='followers',children=['Followers', html.Br(), html.B("{:,.0f}".format(total_followers), className='overview-count')], className='overview-container')
            ], 
            className='overview-wrapper'),

        ]),

        dbc.Row([
            dcc.Markdown('**Follower Growth Trend** ', className='heading1'),
            html.P('Follower Growth shows the increase in the number of your followers. ', className='heading2'),

            dcc.Graph(
                figure=px.line(monthly_followers, x='YearMonth', y='New followers', color_discrete_sequence=['#b51a00'],  markers=True),
                className='followers-graph',
            ),

            dbc.Row([
                dcc.Dropdown(
                    id='audience-dropdown',
                    options=[
                        {'label': 'Job titles', 'value': 'Job titles'},
                        {'label': 'Locations', 'value': 'Locations'},
                        {'label': 'Industries', 'value': 'Industries'},
                        {'label': 'Seniority', 'value': 'Seniority'},
                        {'label': 'Company size', 'value': 'Company size'},
                        {'label': 'Companies', 'value': 'Companies'},
                    ],
                    value='Job titles',
                    className ='audience-dropdown'), 

                dcc.Graph(
                    id='audience-graph',
                    figure = px.bar(demographics[demographics['Top Demographics'] == 'Industries'], x='Percentage', y='Value', title=f'Top Industries by Percentage', labels={'Percentage': 'Percentage (%)', 'Value': 'Top Values'}).update_layout(plot_bgcolor='white'),
                    className='audience-graph')
                ], 
                className='audience-wrapper'
                ),
        ]),

        dbc.Row([
            dcc.Markdown('**Reach/Reactions Trend** ', className='heading1'),
            html.P('Follower Growth shows the increase in the number of your followers. ', className='heading2'),
            dcc.Dropdown(
                id='variable-dropdown',
                options=[
                    {'label': 'Engagements', 'value': 'Engagements'},
                    {'label': 'Impressions', 'value': 'Impressions'},
                    {'label': 'Engagement Rate', 'value': 'EngagementRate'},
                ],
                value='Engagements',
                className='variable-dropdown',),

            dbc.Row([    
                dcc.Graph(
                    id='engagements-graph',
                    figure=px.line(engagements, x='Date', y='Impressions', color_discrete_sequence=['#b51a00']),
                    className='engagements-graph'
                    ),
                
                dbc.Row([
                    dcc.Graph(
                        id='optimal-graph',
                        figure=px.histogram(engagements_by_day, x='DayOfWeek', y='Engagements'),
                        className = 'optimal-graph',
                        ),

                    html.Div([ 
                        html.B('What is the best day of week to post? \n', style={'margin-top': '0px', 'margin-right': '20px', 'Margin-left':'100px','text-align': 'center', 'color': 'gray', 'font-family': 'Helvetica Neue'}),
                        html.B(id ='best-day', children='Reach or Engagement', style={'margin-top': '20px', 'margin-right': '10px', 'font-size':'20px','Margin-left':'10px','align-self': 'center', 'color': 'black', 'font-family': 'Helvetica Neue'}),
                        html.P(id ='optimal-days',children=["Optimal day:", html.B('optimal_day', style={'font-size':'1000px'})], style={'margin-top': '10px', 'margin-right': '20px', 'Margin-left':'10px', 'color': 'Black', 'font-family': 'Arial, sans-serif'})],
                        style={'width':'400px','height':'200px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'10px'})
                ], style = {'width':'500px',})
            ]),  
        ]),
        dcc.Dropdown(
            id='posts-dropdown',
            options=[
                {'label': 'Top engaging posts', 'value': 'Top engaging posts'},
                {'label': 'Top posts by reach', 'value': 'Top posts by reach'},
            ],
            value='Top engaging posts',  
            style={'margin-top': '20px', 'width': '400px', 'margin-left': '40px', 'margin-right': '100px'}),

        dbc.Table(
            id = 'top-posts-preview',
            children=[
                html.Tr([
                    html.Th("Published on"),
                    html.Th("Posts"),
                    html.Th(""),
                    html.Th("Impressions"),
                    html.Th("Reactions"),
                    html.Th("Post Link"),
                    html.Th("Topics"),
                ], style={'background-color': 'lightgray'}),  # Header row styling
            ] +
            # Rows
            [html.Tr([
                html.Td(row['Post publish date'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else html.P('Thumbnail not found.')),
                html.Td(row['Description'],style={'width':'800px', 'margin-left': '10px'}),
                html.Td(row['Impressions'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(row['Engagements'],style={'width':'200px',}),
                html.Td(html.A("View on LI", href=row['Post URL']), style={'width':'200px',}),
                html.Td(row['Topics'],style={'width':'300px',}),
            ], style={'border-bottom': '1px solid gray', 'padding': '10px','width':'1700px'}) for _, row in top_posts_preview_with_topics.head().iterrows()],
            bordered=True,
            responsive=True,
            striped=True,
            hover=True,
            style={'margin': '20px', 'width': '100%'}  # Overall table styling
        ), 
            
    ],fluid = True, 
    className='app-container'
)

@app.callback(
    Output('engagements', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_engagements(start_date, end_date):
    return callbacks.update_engagements(start_date, end_date)

@app.callback(
    Output('impressions', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)    
def update_impressions(start_date, end_date):
    return callbacks.update_impressions(start_date, end_date)

@app.callback(
    Output('engagement_rate', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
) 
def update_engagements_rate(start_date, end_date):
    return callbacks.update_engagements_rate(start_date, end_date)

@app.callback(
    Output('emv_value', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)   
def update_emv(start_date, end_date):   
    return callbacks.update_emv(start_date, end_date)

@app.callback(
    Output('audience-graph', 'figure'),
    [Input('audience-dropdown', 'value')]
)
def update_audience_graph(selected_variable):
    return callbacks.update_audience_graph(selected_variable)

@app.callback(
    Output('engagements-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_engagements_graph(start_date, end_date, selected_variable):
    return callbacks.update_engagements_graph(start_date, end_date, selected_variable)

@app.callback(
    Output('optimal-graph', 'figure'),
    Output('best-day', 'children'),
    Output('optimal-days', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_optimal_graph(start_date, end_date, selected_variable):
    return callbacks.update_optimal_graph(start_date, end_date, selected_variable)


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
